In [1]:
import json

In [2]:
data_name = 'csj'
recog_set = ['dev', 'eval1', 'eval2', 'eval3']
setting = ['withLM']
# setting = ['noLM']
# setting = ['withLM', 'noLM']
nbest = 50

In [ ]:
for s in setting:
    for r in recog_set:
        print(r)
        json_name = f'./data/{dataset}/{s}/{r}_data.json'
        with open(json_name, 'r') as f,\
            open(f'./data/{dataset}/{s}/{r}_hyp.trn', 'w') as hyp,\
            open(f'./data/{dataset}/{s}/{r}_ref.trn', 'w') as ref:
            j = json.load(f)
            for k in j['utts'].keys():
                assert(len(j['utts'][k]['output']) == nbest)
                for i, h in enumerate(j['utts'][k]['output']):
                    # print(j['utts'][k]['output'])
                    seq = [t for t in h['rec_token'].split()]
                    hyp_seq = " ".join(seq[:-1])
                    hyp.write( f'{hyp_seq} ({j["utts"][k]["utt2spk"].replace("-", "_")}-{k}-hyp[{i}])\n' )
                    # hyp.write(hyp_seq + '(' + j["utts"][k]["utt2spk"].replace("-", "_") + '-' + 'hyp' + i + ')')

                    ref_seq = h["token"]
                    ref.write( f'{ref_seq} ({j["utts"][k]["utt2spk"].replace("-", "_")}-{k}-hyp[{i}])\n' )

上面code跑完 執行espnet的
```sclite -r ./ref.trn -h  ./hyp.trn -i rm -o all stdout >  ./result.txt```

In [ ]:
recog_set = ['dev', 'test']

In [3]:
for s in setting:
    for task in recog_set:
        nbest_err = []
        print(f'{s}:{task}')
        with open(f'./data/{data_name}/{s}/{task}_result.txt', 'r') as f:
            temp_err = []
            for i, line in enumerate(f):
                if ('Scores' in line):
                    scores = line.split()[-4:]
                    temp_err.append(scores)
                    if (len(temp_err) == nbest):
                        nbest_err.append(temp_err)
                        temp_err = []
            names = []
            nbest_token = []
            nbest_score = []
            ref_text = []
            with open(f'./data/{data_name}/{s}/{task}_data.json', 'r') as f:
                j = json.load(f)
                for k in j['utts'].keys():
                    token = []
                    score = []
                    for i, h in enumerate(j['utts'][k]['output']):
                        token.append(h["rec_text"])
                        score.append(h['score'])
                    nbest_token.append(token)
                    nbest_score.append(score)
                    ref_text.append(j['utts'][k]['output'][0]['text'])
                    names.append(k)

            dataset = []
            for name, token, score, err, ref in zip(names, nbest_token, nbest_score, nbest_err, ref_text):
                temp_dict = dict()
                temp_dict['name'] = name
                temp_dict['token'] = [t[:-5]  for  t  in token]
                temp_dict['score'] = score
                temp_dict['err'] =  [[int(s) for s in sc] for sc in err]
                temp_dict['ref'] =  ref 
                dataset.append(temp_dict)
            print(len(dataset))
        with open(f"./data/{data_name}/{task}/data_{s}.json", 'w') as f:
            json.dump(dataset, f, ensure_ascii=False, indent = 4)

noLM:dev
4000
noLM:eval1
1272
noLM:eval2
1292
noLM:eval3
1385


## Result trn

In [4]:
dataname = 'aishell'
name = "Audio_Aware"
setting = 'noLM'
recog_set = ['dev', 'test']
nbest = 4
training = 'MD'

# BART

In [5]:
for r in recog_set:
    json_name = f"./src/Correction/data/{dataname}/{setting}/{r}/{nbest}best_rescore_data.json"
    with open(json_name, 'r') as f, \
            open(f'./src/Correction/data/{dataname}/{setting}/{r}/hyp.trn', 'w') as hyp, \
            open(f'./src/Correction/data/{dataname}/{setting}/{r}/ref.trn', 'w') as ref:
        j = json.load(f)
        for k in j['utts'].keys():
                # print(j['utts'][k]['output'])
            hyp_seq = j['utts'][k]['output']['recog_text']
            hyp.write( f'{hyp_seq} ({k})\n' )
            # hyp.write(hyp_seq + '(' + j["utts"][k]["utt2spk"].replace("-", "_") + '-' + 'hyp' + i + ')')
            ref_seq = j['utts'][k]['output']['ref_token']
            ref.write( f'{ref_seq} ({k})\n' )

In [7]:
for r in recog_set:
    json_name = f"./src/Correction/data/{dataname}/{setting}/{r}/{nbest}align/correct_data.json"
    with open(json_name, 'r') as f, \
            open(f'./src/Correction/data/{dataname}/{setting}/{r}/{nbest}align/hyp.trn', 'w') as hyp, \
            open(f'./src/Correction/data/{dataname}/{setting}/{r}/{nbest}align/ref.trn', 'w') as ref:
        j = json.load(f)
        for k in j['utts'].keys():
                # print(j['utts'][k]['output'])
            hyp_seq = j['utts'][k]['hyp']
            hyp.write( f'{hyp_seq} ({k})\n' )
            # hyp.write(hyp_seq + '(' + j["utts"][k]["utt2spk"].replace("-", "_") + '-' + 'hyp' + i + ')')
            ref_seq = j['utts'][k]['ref']
            ref.write( f'{ref_seq} ({k})\n' )

# RescoreBert

In [11]:
for task in recog_set:
    json_name = f"./data/{dataname}/{task}/{training}/{setting}/{nbest}best_rescore_data.json"
    with open(json_name, 'r') as f, \
        open(f'./data/{dataname}/{task}/{training}/{setting}/{nbest}best_hyp.trn', 'w') as hyp, \
        open(f'./data/{dataname}/{task}/{training}/{setting}/{nbest}best_ref.trn', 'w') as ref:
        j = json.load(f)
        for k in j['utts'].keys():
                # print(j['utts'][k]['output'])
            hyp_seq = j['utts'][k]['output']['rec_token']
            hyp.write( f'{hyp_seq} ({k})\n' )
            # hyp.write(hyp_seq + '(' + j["utts"][k]["utt2spk"].replace("-", "_") + '-' + 'hyp' + i + ')')
            ref_seq = j['utts'][k]['output']['text_token']
            ref.write( f'{ref_seq} ({k})\n' )
    print(f'sclite -r ./{nbest}best_hyp.trn -h  ./{nbest}best_ref.trn -i rm -o all stdout >  ./{nbest}best_result.txt')

sclite -r ./50best_hyp.trn -h  ./50best_ref.trn -i rm -o all stdout >  ./50best_result.txt
sclite -r ./50best_hyp.trn -h  ./50best_ref.trn -i rm -o all stdout >  ./50best_result.txt


In [ ]:
for r in recog_set:
    json_name = f'./data/{dataset}_{r}/{name}/rescore_data.json'
    with open(json_name, 'r') as f, open(f'./data/{dataset}_{r}/{name}/hyp_rescore.trn', 'w') as hyp, open(f'./data/{dataset}_{r}/{name}/ref_rescore.trn', 'w') as ref:
        j = json.load(f)
        for k in j['utts'].keys():
                # print(j['utts'][k]['output'])
            hyp_seq = j['utts'][k]['rec_text']
            hyp.write( f'{hyp_seq} ({k})\n' )
            # hyp.write(hyp_seq + '(' + j["utts"][k]["utt2spk"].replace("-", "_") + '-' + 'hyp' + i + ')')
            ref_seq = j['utts'][k]['ref_text']
            ref.write( f'{ref_seq} ({k})\n' )

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

In [ ]:
import json
import torch
# for r in recog_set:
best_weight = 0
best_cer = 100
with open(f"./data/{dataname}/dev/{nbest}best_{training}_rescore_data_{setting}.json") as f:
    data = json.load(f)
    
    for n in range(101):
        weight = n * 0.01
        c = 0
        s = 0
        de = 0
        i = 0
        for d in data:
            score = torch.tensor(d['score'])
            pll = torch.tensor(d['pll'])
            cer = d['err']
                
            result = score + weight * pll

            max_index = torch.argmax(result).item()

            c += cer[max_index][0]
            s += cer[max_index][1]
            de += cer[max_index][2]
            i += cer[max_index][3]
 
        cer = (s + de + i) / (c + s + de)

        if (cer < best_cer):
            best_cer = cer
            best_weight = weight

print(f'best weight:{best_weight} with cer:{best_cer}')

for task in recog_set:
    with open(f"./data/{dataname}/{task}/{nbest}best_{training}_rescore_data_{setting}.json", 'r') as f, \
         open(f'./data/{dataname}/{task}/hyp_mlm.trn', 'w') as h,\
         open(f'./data/{dataname}/{task}/ref_mlm.trn', 'w') as g:
        data = json.load(f)
        for n, d in enumerate(data):
            score = torch.tensor(d['score'])
            pll = torch.tensor(d['pll'])
            cer = d['err']
            ref = d['ref'][5:-5]
            token = d['token']
                
            result = score + weight * pll

            max_index = torch.argmax(result).item()

            best_hyp = token[max_index]
            sep = best_hyp.index(102)
            hyp_seq = tokenizer.convert_ids_to_tokens(best_hyp[1:sep])

            h.write( f'{" ".join(hyp_seq)} ({r}_{n})\n' )
            g.write( f'{" ".join(list(ref))} ({r}_{n})\n')
